In [11]:
import cv2,keyboard,pyautogui as pg, numpy as np,os,gc,sys
from fastai.vision.all import *
import utils
import torch
#import torch.nn as nn
#import torch.nn.functional as F
import torchvision
from torch.utils.data import Dataset, DataLoader
import random

In [12]:
class ArrayDataset(Dataset):
    def __init__(self):
        global direction_count,path_1
        labels=[]
        img_arr=[]
        for i in range(8):
            #screen_name=path_adding_arr[i]+"\screen_"+path_adding_arr[i][1:]+"_"+str(0)+".png"
            #img = cv2.imread(path_1+screen_name)
            #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
            #img_arr = np.array(img)
            for j in range(direction_count[i]):
                screen_name="\\"+directions_str_arr[i]+"\screen_"+directions_str_arr[i]+"_"+str(j)+".png"
                img = cv2.imread(path_1+screen_name)
                #img=torch.tensor(img,device="cuda")
                #img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                img=np.swapaxes(img,0,2)
                img_arr.append(img)
                #this_label=[[0,0,0,0]]
                #this_label[0][i]=1
                #this_label=[0,0,0,0]
                #this_label[i]=1
                this_label=i
                labels.append(np.array(this_label))
        img_arr=np.array(img_arr)
        labels=np.array(labels)
        self.x, self.y = img_arr, labels
        #self.c = 2 # binary label
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, i):
        #j=random.randint(0,len(self))
        return torch.tensor(self.x[i],device="cuda",dtype=torch.float), torch.tensor(self.y[i],device="cuda",dtype=torch.int)

In [13]:
def process_cfg(cfg_file_arr):
    global directions_str_arr
    cfg_file_result=[]
    direction_count_str=['directions']
    direction_count=[]
    path=cfg_file_arr[0][1]
    path_imgs=os.path.join(path,"directions_imgs")
    if os.path.exists(path_imgs)==False:
        os.mkdir(path_imgs)
    for i in directions_str_arr:
        try:
            #direction_count_str.append(str(len(os.listdir(os.path.join(path,i)))))
            direction_count.append(len(os.listdir(os.path.join(path_imgs,i))))
        except:
            os.mkdir(os.path.join(path_imgs,i))
            #direction_count_str.append("0")
            direction_count.append(0)
    cfg_file_result.append(direction_count_str)
    
    screen_arr=[]
    for i in range(1,len(cfg_file_arr[1])):
        if i<5:
            screen_arr.append(int(cfg_file_arr[1][i]))  
    return direction_count,screen_arr,path

In [23]:
root_path=r"Vampire_survivors_bot\Vampire_survivors_cfg.txt"
path_adding=""
posList=[]
#path_adding_arr=["\\right","\\left","\\forward","\\back","\\space","\\nothing"]
directions_str_dict={"r":0,"l":1,"u":2,"d":3,"ru":4,"rd":5,"lu":6,"ld":7}
directions_str_arr=["r","l","u","d","ru","rd","lu","ld"]
cfg_file_arr=[]
#path_1=r"G:\DS\Vampire_survivors"
try:
    file=open(root_path)
    for it, line in enumerate(file):
        line = line.strip()
        cfg_file_arr.append(line.split())
    file.close()
    direction_count,screen_pos_arr,root_path=process_cfg(cfg_file_arr)
    path_1=os.path.join(root_path,"directions_imgs")
    path_2=os.path.join(root_path,"etc","screenshots")
    x1,y1=screen_pos_arr[0],screen_pos_arr[1]
    x2,y2=screen_pos_arr[2],screen_pos_arr[3]
except:
    print("First run grab screen script")
    cv2.waitKey(10000)
    sys.exit()


In [22]:
screen_count=0
cv2.waitKey(5000)
#direction_count=[0,0,0,0,0,0] #r,l,u,d,space,nothing
post_process=1
this_direction_str=""
while True:
    button_pressed=0
    this_direction_str=""
    this_direction=[0,0,0,0]
    screen=pg.screenshot(region=(x1,y1, x2-x1, y2-y1))
    screen=cv2.cvtColor(np.array(screen), cv2.COLOR_BGR2GRAY)
    screen = cv2.resize(screen, (224, 224))
    if post_process:
        screen = cv2.GaussianBlur(screen, (3, 3), 1)
        screen = cv2.Canny(screen, threshold1=120, threshold2=100)
        path=path_1
    else:
        path=path_2
    cv2.imshow("frame",screen)
    if keyboard.is_pressed("right"):
        this_direction[0]=1
    if keyboard.is_pressed("left"):
        this_direction[1]=1
    if keyboard.is_pressed("up"):
        this_direction[2]=1
    if keyboard.is_pressed("down"):
        this_direction[3]=1
    #if keyboard.is_pressed("v"): #reserved for space button
        #this_direction[4]=1
    for i in range(4):
        if this_direction[i]:
            button_pressed=1
            this_direction_str+=directions_str_arr[i]
        
    if keyboard.is_pressed("escape"):
        break
    #else:
        #path_adding="\\nothing" #reserved for no keys pressed
        #this_direction[5]=1
    if (button_pressed):
        try:
            this_direction_idx=directions_str_dict[this_direction_str]
            screen_name="\\"+this_direction_str+"\screen_"+this_direction_str+"_"+str(direction_count[this_direction_idx])+".png"   
            cv2.imwrite(path+screen_name, screen)
            direction_count[this_direction_idx]+=1
        except:
            pass
    #img = cv2.imread("screen.png")
    #if (keyboard.is_pressed("f")):
        #cv2.waitKey("f")
    cv2.waitKey(20)
cv2.destroyWindow('frame')
str(direction_count)

'[0, 0, 0, 0, 0, 0, 0, 0]'

In [ ]:
images_data=ArrayDataset()
data_size=len(images_data)
val_size_rel=0.1 #train-test split part
val_size=round(data_size*val_size_rel)
train_data,val_data=torch.utils.data.random_split(images_data, [data_size-val_size, val_size])
dls = DataLoaders.from_dsets(train_data,val_data,bs=8,device=torch.device('cuda'))
learner = vision_learner(dls, resnet18,n_out=8,metrics=error_rate, loss_func=CrossEntropyLossFlat())


In [ ]:
learner.fine_tune(2, base_lr=5.0e-03)

In [ ]:
learner.save(root_path+"\\trained_model")
sys.exit()

In [ ]:
#img = cv2.imread(path_2+"\\back\screen_back_1568.png")
#img=np.swapaxes(img,0,2)
#img=torch.tensor(img,device="cuda",dtype=torch.float)
#img.shape

#imgBlur = cv2.GaussianBlur(img, (3, 3), 1)
#flag=0
#while flag==0:
#    for i in range(121,122,5):
#        for j in range(99,100,5):
#            image = cv2.Canny(imgBlur, threshold1=i, threshold2=j)
#            cv2.imshow("frame",image)
#            cv2.imshow("frame_2",img)
#            if keyboard.is_pressed("escape"):
#                flag=1
#                break
#            cv2.waitKey(50)
#cv2.destroyWindow('frame')
#cv2.destroyWindow('frame_2')